# Basic Molecular Dynamics Simulation of Villin N68H in Water with OpenMM

## Prerequisites

Before running this notebook, ensure you have OpenMM installed:
```bash
conda create --platform osx-arm64 -n openmm-env python=3.13 conda-forge::openmm
conda activate openmm-env
```

You'll also need a PDB file named `villin-n68h_solv.pdb` in your working directory. This should contain the initial coordinates of your molecular system.

## Complete Code

In [ ]:
# Import necessary modules
from sys import stdout
from openmm import app
from openmm import unit
from openmm import LangevinMiddleIntegrator

# Load molecular structure
pdb = app.PDBFile("villin-n68h_solv.pdb")

# Define force field parameters
forcefield = app.ForceField(
    'amber19-all.xml', 'amber19/tip3pfb.xml'
)

# Create physical system with simulation parameters
system = forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=app.PME,
    nonbondedCutoff=1*unit.nanometer,
    constraints=app.HBonds
)

# Set up Langevin integrator for constant temperature dynamics
integrator = LangevinMiddleIntegrator(
    300*unit.kelvin, 1/unit.picosecond, 0.004*unit.picoseconds
)

# Create simulation object and set initial positions
simulation = app.Simulation(
    pdb.topology, system, integrator
)
simulation.context.setPositions(pdb.positions)

# Minimize energy to remove bad contacts
simulation.minimizeEnergy()

# Set up trajectory and progress reporting
simulation.reporters.append(
    app.DCDReporter('villin-n68h_solv_trj.dcd', 1000)
)
simulation.reporters.append(
    app.StateDataReporter(
        stdout, 1000,
        step=True, 
        potentialEnergy=True, 
        temperature=True
    )
)

# Run molecular dynamics simulation
simulation.step(10000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-143943.44927291677,291.5752260721319
2000,-142703.57036666677,297.5115860040943
3000,-142326.38481979177,303.94724375957907
4000,-141324.17974166677,305.42412568909475
5000,-141733.56450729177,298.5153432952187
6000,-141887.39360885427,302.54082609242334
7000,-141235.52056197927,305.0134360889396
8000,-141433.27739791677,304.12855302510394
9000,-142119.27935104177,303.3014125957956
10000,-141940.56548385427,300.51936464359557
